<a href="https://colab.research.google.com/github/reeshabhkumarranjan/ML-Course-Project/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import numpy as np
import glob
from keras import Sequential
import sklearn, sklearn.model_selection
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Activation
from keras.utils import to_categorical, plot_model

In [94]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

# loading data
colab_directory = '/content/drive/My Drive/Colab Notebooks/ML Course Project/'
x = np.load(colab_directory + "x.npy")
y = np.load(colab_directory + "y.npy")
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# common variables
epochs = 3
showWorking = False
showWorkingAverage = True
batchSize = 10
dropoutRate = 0.5
activationFunction = 'relu'
iterations = 5
model_description_mode = False
iterations = 1 if model_description_mode else iterations
numFilters = 30

In [96]:
# LSTM model
accuracy_average = 0
for i in range(iterations):
	model_lstm = Sequential([
		LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2])),
		# Dropout(dropoutRate),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax'),
	])
 
	# save the architecture of the model
	if model_description_mode:
		print("LSTM")
		plot_model(model_lstm, colab_directory + 'model_lstm.png')
		print(model_lstm.summary())	
	else:
		model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
		model_lstm.fit(x_train, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
		accuracy_average += model_lstm.evaluate(x_test, y_test, batch_size=batchSize, verbose=showWorking)[1]
		if showWorkingAverage:
			print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))
if not model_description_mode:
	accuracy_average /= iterations
	print("Accuracy of LSTM model on test: ", accuracy_average)
print()

1: 0.590000006556511
2: 0.6300000056624413
3: 0.7233333359162013
4: 0.7400000013411046
5: 0.7799999994039536
Accuracy of LSTM model on test:  0.7799999994039536



In [97]:
# CNN-LSTM
accuracy_average = 0
for i in range(iterations):
	trainX = x_train.reshape((x_train.shape[0], 3, 20, x_train.shape[2]))
	testX = x_test.reshape((x_test.shape[0], 3, 20, x_test.shape[2]))

	# conv = Conv1D(filters=numFilters, kernel_size=3, activation=activationFunction)
	conv = Conv1D(filters=numFilters, kernel_size=3)
	# conv2 = Conv1D(filters=64, kernel_size=3, activation=activationFunction)
	model_lstm_cnn = Sequential([
		TimeDistributed(conv, input_shape=(None, 20, x_train.shape[2])),
		TimeDistributed(Activation(activation=activationFunction)),
		# TimeDistributed(conv2),
		# TimeDistributed(Dropout(dropoutRate)),
		TimeDistributed(MaxPooling1D(pool_size=2)),
		TimeDistributed(Flatten()),
		LSTM(200),
		# Dropout(dropoutRate),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]), # y is one-hot encoded, hence its number of columns is number of classes
		Activation('softmax')
	])
 
	if model_description_mode:
		# save the architecture of the model
		print("CNN-LSTM")
		plot_model(model_lstm_cnn, colab_directory + 'model_lstm_cnn.png')
		print(model_lstm_cnn.summary())
	
	else:
		model_lstm_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
		model_lstm_cnn.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)

		accuracy_average += model_lstm_cnn.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)[1]
		if showWorkingAverage:
			print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))

if not model_description_mode:
	accuracy_average /= iterations
	print("Accuracy of LSTM-CNN model on test: ", accuracy_average)
print()

1: 0.8200000047683715
2: 0.8749999970197677
3: 0.8966666599114737
4: 0.9074999913573265
5: 0.8939999926090241
Accuracy of LSTM-CNN model on test:  0.8939999926090241



In [98]:
# LSTM + conv
accuracy_average = 0
for i in range(iterations):
	trainX = x_train.reshape((x_train.shape[0], 3, 1, 20, x_train.shape[2]))
	testX = x_test.reshape((x_test.shape[0], 3, 1, 20, x_test.shape[2]))

	model_lstm_conv = Sequential([
		ConvLSTM2D(filters=numFilters, kernel_size=(1,3), input_shape=(3, 1, 20, x_train.shape[2])),
		Activation(activationFunction),
		# Dropout(dropoutRate),
		Flatten(),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax')
	])
 
	if model_description_mode:
		print("Convolutional LSTM")
		# save the architecture of the model
		plot_model(model_lstm_conv, colab_directory + 'model_lstm_conv.png')
		print(model_lstm_conv.summary())
	
	else: 
		model_lstm_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
		model_lstm_conv.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)

		accuracy_average += model_lstm_conv.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)[1]
		if showWorkingAverage:
			print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))
if not model_description_mode:
	accuracy_average /= iterations
	print("Accuracy of LSTM-Conv model on test: ", accuracy_average)
print()

1: 0.8899999916553497
2: 0.9049999922513962
3: 0.8966666599114737
4: 0.9049999922513963
5: 0.8999999916553498
Accuracy of LSTM-Conv model on test:  0.8999999916553498

